In [10]:
import requests
import time
import pandas as pd
import talib
import numpy as np
import plotly.graph_objects as plt

# Data Retrival
BTCUSDT Kline trading data with time interval = 15min

In [11]:
URL = "https://www.binancezh.top/fapi/v1/klines?symbol=BTCUSDT&interval=15m&limit=1500"
headers = {'user-agent': 'Chrome/102.0'}
# proxies = {'http': 'http://192.168.0.8', 'https': 'http://192.168.0.8'} # IP address? Server address?
r = requests.get(url=URL, headers=headers)
results = r.json()

In [12]:
df = []
for res in results:
    res[0] = time.strftime("%Y-%m-%d %H:%M", time.localtime(int(res[0])/1000))
    res[6] = time.strftime("%Y-%m-%d %H:%M", time.localtime(int(res[6])/1000))
    res = res[:-1]
    df.append(res)

In [13]:
df = pd.DataFrame(df, columns=['OpenTm','Open','High','Low','Close','Volume','ClsTm','QuoteVl','NTrades','TbVol','TbQV'])
df.to_csv('BTCUSDT.csv', sep=',', encoding='utf-8')

# SAR Calculation
Generate SAR indicator with TA-Lib.
## Formula
Rising $SAR_n = SAR_{n-1}+AF_n*[EP_{n-1}-SAR_{n-1}]$ \
Falling $SAR_n = SAR_{n-1}+AF_n*[SAR_{n-1}-EP_{n-1}]$ \
where AF = Acceleration Factor, EP = Extreme Point \
AF starts at 0.02 and increases by 0.02, up to a maximum of 0.2, each time the extreme point makes a new low (falling
SAR) or high (rising SAR). EP is the lowest low in the current downtrend(falling SAR) or the highest high in the current uptrend(rising SAR).

In [14]:
SAR = talib.SAR(np.array(df.High.astype('float64')), np.array(df.Low.astype('float64')))
df['SAR'] = SAR
# df.to_csv('BTCUSDT_SAR.csv', sep=',', encoding='utf-8')

# Strategy Construction
Buy-Sell Signal

In [15]:
df.Close = df.Close.astype('float64')
df['Signal'] = 0 
for i in range(1, len(df)):
    if df.loc[i, 'SAR'] < df.loc[i, 'Close']:
        df.loc[i, 'Signal'] = 1
    else:
        df.loc[i, 'Signal'] = -1
df.Signal = df.Signal.shift(1)
df.head()

,OpenTm,Open,High,Low,Close,Volume,ClsTm,QuoteVl,NTrades,TbVol,TbQV,SAR,Signal
0,2022-06-03 20:45,29574.40,29662.20,29441.70,29520.2,9829.319,2022-06-03 20:59,290551152.74482,85732,4721.008,139602342.21827,NaN,NaN
1,2022-06-03 21:00,29520.20,29644.50,29417.70,29575.3,9554.540,2022-06-03 21:14,282339360.03654,79789,4770.704,141003284.01250,29662.200,0.0
2,2022-06-03 21:15,29573.80,29685.20,29562.00,29562.1,7324.195,2022-06-03 21:29,216954815.36659,53577,3518.167,104207346.89582,29417.700,-1.0
3,2022-06-03 21:30,29562.00,29764.30,29528.70,29595.1,11409.761,2022-06-03 21:44,338527100.79570,89890,5960.071,176866539.78669,29417.700,1.0
4,2022-06-03 21:45,29595.10,29713.70,29552.50,29705.0,5462.494,2022-06-03 21:59,161852305.61185,56250,2892.329,85705049.27477,29431.564,1.0


In [16]:
df.Open = df.Open.astype('float64')
df['Ret'] = 0
for i in range(len(df)):
    df.loc[i, 'Ret'] = df.loc[i,'Close']/df.loc[i,'Open'] - 1
df['StrategyRet'] = 0
for i in range(len(df)):
    df.loc[i, 'StrategyRet'] = df.loc[i,'Signal'] * df.loc[i,'Ret']
df.head()

,OpenTm,Open,High,Low,Close,Volume,ClsTm,QuoteVl,NTrades,TbVol,TbQV,SAR,Signal,Ret,StrategyRet
0,2022-06-03 20:45,29574.4,29662.20,29441.70,29520.2,9829.319,2022-06-03 20:59,290551152.74482,85732,4721.008,139602342.21827,NaN,NaN,-0.001833,NaN
1,2022-06-03 21:00,29520.2,29644.50,29417.70,29575.3,9554.540,2022-06-03 21:14,282339360.03654,79789,4770.704,141003284.01250,29662.200,0.0,0.001867,0.000000
2,2022-06-03 21:15,29573.8,29685.20,29562.00,29562.1,7324.195,2022-06-03 21:29,216954815.36659,53577,3518.167,104207346.89582,29417.700,-1.0,-0.000396,0.000396
3,2022-06-03 21:30,29562.0,29764.30,29528.70,29595.1,11409.761,2022-06-03 21:44,338527100.79570,89890,5960.071,176866539.78669,29417.700,1.0,0.001120,0.001120
4,2022-06-03 21:45,29595.1,29713.70,29552.50,29705.0,5462.494,2022-06-03 21:59,161852305.61185,56250,2892.329,85705049.27477,29431.564,1.0,0.003713,0.003713


In [17]:
df['Cumret'] = np.cumprod(1+df.Ret)
df['StrategyCumret'] = np.cumprod(1+df.StrategyRet)
df.head()

,OpenTm,Open,High,Low,Close,Volume,ClsTm,QuoteVl,NTrades,TbVol,TbQV,SAR,Signal,Ret,StrategyRet,Cumret,StrategyCumret
0,2022-06-03 20:45,29574.4,29662.20,29441.70,29520.2,9829.319,2022-06-03 20:59,290551152.74482,85732,4721.008,139602342.21827,NaN,NaN,-0.001833,NaN,0.998167,NaN
1,2022-06-03 21:00,29520.2,29644.50,29417.70,29575.3,9554.540,2022-06-03 21:14,282339360.03654,79789,4770.704,141003284.01250,29662.200,0.0,0.001867,0.000000,1.000030,1.000000
2,2022-06-03 21:15,29573.8,29685.20,29562.00,29562.1,7324.195,2022-06-03 21:29,216954815.36659,53577,3518.167,104207346.89582,29417.700,-1.0,-0.000396,0.000396,0.999635,1.000396
3,2022-06-03 21:30,29562.0,29764.30,29528.70,29595.1,11409.761,2022-06-03 21:44,338527100.79570,89890,5960.071,176866539.78669,29417.700,1.0,0.001120,0.001120,1.000754,1.001516
4,2022-06-03 21:45,29595.1,29713.70,29552.50,29705.0,5462.494,2022-06-03 21:59,161852305.61185,56250,2892.329,85705049.27477,29431.564,1.0,0.003713,0.003713,1.004470,1.005235


In [18]:
Cumret=plt.Scatter(x=df.index, y=df.Cumret, mode='lines', name='Cumulative Returns')
StCumret=plt.Scatter(x=df.index, y=df.StrategyCumret, mode='lines', name='SAR Cumulative Returns')
fig=plt.Figure([Cumret, StCumret])
fig.update_layout(title = 'SAR Strategy', xaxis_title = 'Year', yaxis_title = 'Returns')
fig.show()